In [1]:
import os

# --- Define Paths ---
install_dir = "/content/drive/MyDrive/Colab_Software/GROMACS"
gromacs_env_script = os.path.join(install_dir, "gromacs_env.sh")
gmx_executable_path = os.path.join(install_dir, "bin/gmx")
conda_env_name = "gmxmmpbsa"

# 1. Mount Google Drive
print("Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

# 2. CRITICAL FIX: Re-apply execute permission for GROMACS (Solves 'Permission denied')
print(f"Restoring execute permission for GROMACS: {gmx_executable_path}")
!chmod +x {gmx_executable_path}

# 3. Source GROMACS environment variables (for the current shell/Python process)
print(f"Sourcing GROMACS environment script: {gromacs_env_script}")
!source {gromacs_env_script}

# 4. Re-initialize Condacolab (Required for gmxMMPBSA)
print("\nInstalling condacolab. This will RESTART the runtime.")
print("After the restart, run the next cell (Step 2).")
!pip install -q condacolab
import condacolab
condacolab.install()

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Restoring execute permission for GROMACS: /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Sourcing GROMACS environment script: /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh

Installing condacolab. This will RESTART the runtime.
After the restart, run the next cell (Step 2).
⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...


In [2]:
import os
import subprocess

conda_env_name = "gmxmmpbsa"

# Ensure the Conda shell is initialized for this Python session (redundant but safe)
!source /usr/local/etc/profile.d/conda.sh

# --- Creating 'gmxmmpbsa' Conda Environment and Installing Tools ---
print(f"\n--- Creating '{conda_env_name}' Conda Environment and Installing Tools ---")

# 1. Create Conda Environment
!conda create -n {conda_env_name} python=3.9 -y

# 2. Install AmberTools & Compilers
print("\nInstalling AmberTools, Compilers, and MPI...")
!conda run -n {conda_env_name} conda install -c conda-forge ambertools compilers mpi4py -y

# 3. Install All Python Analysis Tools
print("\nInstalling gmx_MMPBSA and other Python tools...")
!conda run -n {conda_env_name} pip install gmx_MMPBSA acpype MDAnalysis prolif pandas matplotlib seaborn

print(f"\n--- ✅ Installation of Analysis Tools Finished in Current Runtime ---")


--- Creating 'gmxmmpbsa' Conda Environment and Installing Tools ---
Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/gmxmmpbsa

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       hda65f42_8         254 KB  conda-forge
    ca-certificates-2025.10.5  |       hbd8a1cb_0         152 KB  conda-forge
    ld_impl_linux-64-2.44      |       ha97dd6f_2         730 KB  conda-forge
    libexpat-2.7.1             |       hecca717_0          73 KB  conda-forge
    libffi-3.5.2               |       h9ec8514_0          56 KB  conda-forge
    libgcc-1

In [3]:
print("--- Installing ImageMagick and DFT Dependencies ---")
!apt-get update -y
!apt-get install -y libxc-dev liblapack-dev libblas-dev imagemagick

# Verify ImageMagick (convert)
print("\n--- 4. ImageMagick Verification ---")
!convert -version

print("\n--- ✅ All tools are now 100% ready for use! ---")

--- Installing ImageMagick and DFT Dependencies ---
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,086 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.

In [1]:
# --- Verification (Run after the installation cell above) ---
import os

# Define paths
install_dir = "/content/drive/MyDrive/Colab_Software/GROMACS"
gromacs_env_script = os.path.join(install_dir, "gromacs_env.sh")
gmx_executable_path = os.path.join(install_dir, "bin/gmx")
conda_env_name = "gmxmmpbsa"

# CRITICAL FIX 1: GROMACS Environment Setup
# Re-fix and Re-source the GROMACS path
!chmod +x {gmx_executable_path}

# CRITICAL FIX 2: CONDA Environment Re-Initialization and Verification
print("\n--- Running final verification via combined bash script... ---")

# Execute the combined verification script (using the correct f-string syntax)
script_content = f"""#!/bin/bash
# 1. Manually source the Conda initialization script
source /usr/local/etc/profile.d/conda.sh

# 2. Source the GROMACS environment script
source {gromacs_env_script}

echo ""
echo "--- 1. GROMACS Verification ---"
gmx -version

echo ""
echo "--- 2. gmxMMPBSA Verification ---"
conda run -n {conda_env_name} gmx_MMPBSA -v

echo ""
echo "--- 3. AmberTools (antechamber) Verification ---"
conda run -n {conda_env_name} antechamber -h

echo ""
echo "--- 4. ImageMagick Verification ---"
convert -version
"""

# Write the script content to the file
with open("final_verification_check.sh", "w") as f:
    f.write(script_content)

# Execute the combined verification script
!bash final_verification_check.sh


--- Running final verification via combined bash script... ---

--- 1. GROMACS Verification ---
                         :-) GROMACS - gmx, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content
Command line:
  gmx -version

GROMACS version:     2025.3
Precision:           mixed
Memory model:        64 bit
MPI library:         thread_mpi
OpenMP support:      enabled (GMX_OPENMP_MAX_THREADS = 128)
GPU support:         CUDA
NBNxM GPU setup:     super-cluster 2x2x2 / cluster 8 (cluster-pair splitting on)
SIMD instructions:   AVX2_256
CPU FFT library:     fftw-3.3.10-sse2-avx-avx2-avx2_128
GPU FFT library:     cuFFT
Multi-GPU FFT:       none
RDTSCP usage:        enabled
TNG support:         enabled
Hwloc support:       disabled
Tracing support:     disabled
C compiler:          /usr/bin/cc GNU 11.4.0
C compiler flags:    -fexcess-precision=fast -funroll-all-loops -mavx2 -mfma -Wno-m

In [2]:
%%bash

# Define paths (optional, but clean)
GROMACS_INSTALL_DIR="/content/drive/MyDrive/Colab_Software/GROMACS"
ENV_SCRIPT="${GROMACS_INSTALL_DIR}/gromacs_env.sh"

# Source environment for this bash session
source ${ENV_SCRIPT}

# Now gmx should be available
gmx --version

                         :-) GROMACS - gmx, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content
Command line:
  gmx --version

GROMACS version:     2025.3
Precision:           mixed
Memory model:        64 bit
MPI library:         thread_mpi
OpenMP support:      enabled (GMX_OPENMP_MAX_THREADS = 128)
GPU support:         CUDA
NBNxM GPU setup:     super-cluster 2x2x2 / cluster 8 (cluster-pair splitting on)
SIMD instructions:   AVX2_256
CPU FFT library:     fftw-3.3.10-sse2-avx-avx2-avx2_128
GPU FFT library:     cuFFT
Multi-GPU FFT:       none
RDTSCP usage:        enabled
TNG support:         enabled
Hwloc support:       disabled
Tracing support:     disabled
C compiler:          /usr/bin/cc GNU 11.4.0
C compiler flags:    -fexcess-precision=fast -funroll-all-loops -mavx2 -mfma -Wno-missing-field-initializers -O3 -DNDEBUG
C++ compiler:        /usr/bin/c++ GNU 11.4.0
C++ compiler

In [3]:
# Define the target directory path
target_dir = "/content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF"

# Create the directory if it doesn't exist (ensures your path is valid)
import os
os.makedirs(target_dir, exist_ok=True)

# Change the current working directory using the %cd magic command
print(f"Changing directory to: {target_dir}")
%cd {target_dir}

# Verify the change
!pwd
!ls -F # List contents to confirm you are in the right place

Changing directory to: /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF
/content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF
/content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF
control/		     gmx_MMPBSA.log  lupeol/
final_verification_check.sh  hedragenin/     maslinic_acid/


In [4]:
# Check GPU CUDA core information
!nvidia-smi -q


==============NVSMI LOG==============

Timestamp                                 : Wed Oct 22 20:06:28 2025
Driver Version                            : 550.54.15
CUDA Version                              : 12.4

Attached GPUs                             : 1
GPU 00000000:00:04.0
    Product Name                          : Tesla T4
    Product Brand                         : NVIDIA
    Product Architecture                  : Turing
    Display Mode                          : Enabled
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    Addressing Mode                       : None
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
    Serial Number                 

In [5]:
# Check CPU information
import multiprocessing

print(f"Number of CPU cores available: {multiprocessing.cpu_count()}")

Number of CPU cores available: 2


In [6]:
# Check GPU information (if available)
import tensorflow as tf

gpu_available = tf.config.list_physical_devices('GPU')
if gpu_available:
    print("GPU is available.")
    print("GPU details:")
    for gpu in gpu_available:
        print(gpu)
else:
    print("No GPU available.")

GPU is available.
GPU details:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [7]:
!date

Wed Oct 22 08:06:34 PM UTC 2025


In [8]:
!ls

control			     gmx_MMPBSA.log  lupeol
final_verification_check.sh  hedragenin      maslinic_acid


!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh &&

To source the gmx command

In [9]:
%cd "/content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin"

/content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin


In [10]:
!ls

 ANTECHAMBER_AC.AC		 '#md_25_50.xtc.1#'
 ANTECHAMBER_AC.AC0		  md2.mdp
 ANTECHAMBER_AM1BCC.AC		  md_50_75.cpt
 ANTECHAMBER_AM1BCC_PRE.AC	  md_50_75.part0003.edr
 ANTECHAMBER_BOND_TYPE.AC	 '#md_50_75.part0003.edr.1#'
 ANTECHAMBER_BOND_TYPE.AC0	  md_50_75.part0003.gro
 ATOMTYPE.INF			  md_50_75.part0003.log
 box.gro			 '#md_50_75.part0003.log.1#'
 cgenff_charmm2gmx_py3_nx1.py	 '#md_50_75.part0003.log.2#'
 complex_1h6u_hed_137633443.pdb   md_50_75.part0003_prev.cpt
 complex.gro			  md_50_75.part0003.xtc
 em.edr				 '#md_50_75.part0003.xtc.1#'
 em.gro				  md_50_75.part0004.gro
 em.log				  md_50_75.part0005.gro
 em.mdp				  md_50_75_prev.cpt
 em.tpr				  md_50_75.tpr
 em.trr				 '#md_50_75.tpr.1#'
 energy.xvg			 '#md_50_75.tpr.2#'
'Gromacs Codes'			  md_75_100.cpt
 index.ndx			  md_75_100.part0001_prev.cpt
 interaction_analysis.py	 '#md_75_100.part0002.edr.1#'
 ions.gro			 '#md_75_100.part0002.log.1#'
 ions.mdp			 '#md_75_100.part0002.xtc.1#'
 ions.tpr			  md_75_100.part0004.edr
 lig.acpy

In [ ]:
!perl sort_mol2_bonds.pl lig.mol2 lig.mol2

Found 79 atoms in the molecule, with 83 bonds.


In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx pdb2gmx -f protein.pdb -o protein.gro -ignh

                     :-) GROMACS - gmx pdb2gmx, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx pdb2gmx -f protein.pdb -o protein.gro -ignh

Note that more recent versions of the CHARMM force field may be downloaded from
http://mackerell.umaryland.edu/charmm_ff.shtml#gromacs.

Select the Force Field:

From '/content/drive/MyDrive/Colab_Software/GROMACS/share/gromacs/top':

 1: AMBER03 protein, nucleic AMBER94 (Duan et al., J. Comp. Chem. 24, 1999-2012, 2003)

 2: AMBER94 force field (Cornell et al., JACS 117, 5179-5197, 1995)

 3: AMBER96 protein, nucleic AMBER94 (Kollman et al., Acc. Chem. Res. 29, 461-469, 1996)

 4: AMBER99 protein, nucleic AMBER94 (Wang et al., J. Comp. Chem. 21, 1049-1074, 2000)

 5: AMBER99SB protein, nucleic AMBER94 (Hornak et al., Proteins 65, 712-725, 2006)

In [ ]:
!pip install acpype

  Using cached acpype-2023.10.27-py3-none-any.whl.metadata (12 kB)
Using cached acpype-2023.10.27-py3-none-any.whl (28.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 30.7 MB/s eta 0:00:00


In [ ]:
conda_env_name

'gmxmmpbsa'

In [ ]:
print("--- 1. Running Antechamber ---")
!conda run -n {conda_env_name} antechamber -i lig.mol2 -fi mol2 -o lig_fixed.mol2 -fo mol2 -c bcc -s 2

--- 1. Running Antechamber ---

Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for mol2 File --
   Status: pass
Info: Finished reading file (lig.mol2); atoms read (79), bonds read (83).
Info: Determining atomic numbers from atomic symbols which are case sensitive.
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Running: /usr/local/envs/gmxmmpbsa/bin/bondtype -j full -i ANTECHAMBER_BOND_TYPE.AC0 -o ANTECHAMBER_BOND_TYPE.AC -f ac

Running: /usr/local/envs/gmxmmpbsa/bin/atomtype -i ANTECHAMBER_AC.AC0 -

In [ ]:
print("\n--- 2. Running Parmchk2 ---")
!conda run -n {conda_env_name} parmchk2 -i lig_fixed.mol2 -f mol2 -o lig.frcmod -s gaff


--- 2. Running Parmchk2 ---


In [ ]:
# Ensure the Conda shell is initialized for this Python session
!source /usr/local/etc/profile.d/conda.sh

# 4. Generate GROMACS Topology with acpype
print("\nGenerating GROMACS Topology with acpype...")
!conda run -n gmxmmpbsa acpype -i lig_fixed.mol2 -b lig -c user -o gmx


Generating GROMACS Topology with acpype...
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2023.10.27 (c) 2025 AWSdS |
==> Reading user's charges from mol2 file...
==> ... charge set to 0
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
==> * Tleap OK *
==> Removing temporary files...
==> Using OpenBabel v.3.1.0

==> Writing GROMACS files

==> Disambiguating lower and uppercase atomtypes in GMX top file, even if identical.

==> Writing GMX dihedrals for GMX 4.5 and higher.

==> Writing pickle file lig.pkl
==> Removing temporary files...
Total time of execution: 1s

*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in ReadMolecule
  This Mol2 file is non-standard. Problem with molecule: complex_1h6u_hed_137633443.pdb Cannot interpret atom types correctly, instead attempting to interpret atom type: c3 as elements instead.
*** Open Babel Warning  in Translat

In [ ]:
import os
import shutil

# Define source and destination paths
source_itp = "lig.acpype/lig_GMX.itp"
source_gro = "lig.acpype/lig_GMX.gro"
destination_dir = "." # Current directory

# Copy ligand files
print(f"Copying {source_itp} to {destination_dir}")
shutil.copy(source_itp, destination_dir)

print(f"Copying {source_gro} to {destination_dir}")
shutil.copy(source_gro, destination_dir)

print("Ligand files copied successfully.")

Copying lig.acpype/lig_GMX.itp to .
Copying lig.acpype/lig_GMX.gro to .
Ligand files copied successfully.


In [ ]:
# Edit topol.top
topol_file = "topol.top"

# Read the original topol.top content
with open(topol_file, "r") as f:
    lines = f.readlines()

# Find the line after which to include the ligand topology
# This assumes the forcefield include is present and in the format like #include "amber99sb-ildn.ff/forcefield.itp"
insert_index = -1
for i, line in enumerate(lines):
    if line.strip().startswith("#include") and "forcefield.itp" in line:
        insert_index = i + 1
        break

if insert_index != -1:
    # Insert the ligand topology include and molecule definition
    ligand_include = '; Include ligand topology\n#include "lig_GMX.itp"\n'
    ligand_molecule = '\n[ molecules ]\n; Compound        #mols\nProtein_chain_A     1\nlig                 1\n'

    # Remove existing [ molecules ] section if it exists, to avoid duplicates
    molecules_section_start = -1
    for i, line in enumerate(lines):
        if line.strip() == '[ molecules ]':
            molecules_section_start = i
            break

    if molecules_section_start != -1:
        # Find the end of the molecules section (or end of file)
        molecules_section_end = len(lines)
        for i in range(molecules_section_start + 1, len(lines)):
            if line.strip().startswith('[') and not line.strip() == '[ molecules ]':
                 molecules_section_end = i
                 break
        lines = lines[:molecules_section_start] + lines[molecules_section_end:]


    # Insert the ligand include statement after the forcefield include
    lines.insert(insert_index, ligand_include)

    # Add the new molecules section at the end
    lines.extend(ligand_molecule.splitlines(keepends=True))


    # Write the modified content back to topol.top
    with open(topol_file, "w") as f:
        f.writelines(lines)

    print(f"Successfully modified {topol_file}")
else:
    print(f"Could not find the forcefield include in {topol_file}. Please manually edit the file.")

Successfully modified topol.top


In [ ]:
def combine_gro_files(protein_gro, ligand_gro, output_gro):
    """Combines protein and ligand .gro files."""

    with open(protein_gro, 'r') as f_prot:
        prot_lines = f_prot.readlines()

    with open(ligand_gro, 'r') as f_lig:
        lig_lines = f_lig.readlines()

    # Extract coordinate lines (skip header and box vector line)
    # Assuming standard .gro format: Title, num_atoms, coordinates, box_vectors
    prot_coords = prot_lines[2:-1]
    lig_coords = lig_lines[2:-1]

    # Get number of atoms
    num_prot_atoms = int(prot_lines[1].strip())
    num_lig_atoms = int(lig_lines[1].strip())
    total_atoms = num_prot_atoms + num_lig_atoms

    # Get box vectors from protein file (assuming they are the same)
    box_vectors = prot_lines[-1]

    # Write combined .gro file
    with open(output_gro, 'w') as f_out:
        f_out.write(prot_lines[0]) # Write title line
        f_out.write(f"{total_atoms}\n") # Write updated atom count
        f_out.writelines(prot_coords) # Write protein coordinates
        f_out.writelines(lig_coords) # Write ligand coordinates
        f_out.write(box_vectors) # Write box vectors

    print(f"Successfully combined {protein_gro} and {ligand_gro} into {output_gro}")

# Define input and output filenames
protein_gro_file = "protein.gro"
ligand_gro_file = "lig_GMX.gro"
complex_gro_file = "complex.gro"

# Run the function to combine files
combine_gro_files(protein_gro_file, ligand_gro_file, complex_gro_file)

Successfully combined protein.gro and lig_GMX.gro into complex.gro


In [ ]:
%%bash

# Define paths (optional, but good practice)
GROMACS_INSTALL_DIR="/content/drive/MyDrive/Colab_Software/GROMACS"
ENV_SCRIPT="${GROMACS_INSTALL_DIR}/gromacs_env.sh"

# Source the GROMACS environment script and run the command
source ${ENV_SCRIPT} && gmx editconf -f complex.gro -o box.gro -c -d 1.0 -bt triclinic

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 4724 atoms
Volume: 192.251 nm^3, corresponds to roughly 86500 electrons
No velocities found
    system size :  4.209  5.426  8.418 (nm)
    center      : -0.085  0.339  6.097 (nm)
    box vectors :  4.209  5.427  8.417 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  : 192.25               (nm^3)
    shift       :  3.189  3.374 -0.888 (nm)
new center      :  3.105  3.713  5.209 (nm)
new box vectors :  6.209  7.426 10.418 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  : 480.35               (nm^3)


                     :-) GROMACS - gmx editconf, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx editconf -f complex.gro -o box.gro -c -d 1.0 -bt triclinic


GROMACS reminds you: "Exactly" (Pulp Fiction)



In [ ]:
%%bash

# Define paths (optional, but good practice)
GROMACS_INSTALL_DIR="/content/drive/MyDrive/Colab_Software/GROMACS"
ENV_SCRIPT="${GROMACS_INSTALL_DIR}/gromacs_env.sh"

# Source the GROMACS environment script and run the command
source ${ENV_SCRIPT} && gmx solvate -cp box.gro -cs spc216.gro -p topol.top -o solvated.gro


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. (1964)
DOI: 10.1021/j100785a001
-------- -------- --- Thank You --- -------- --------

Adding line for 14336 solvent molecules with resname (SOL) to topology file (topol.top)


                     :-) GROMACS - gmx solvate, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx solvate -cp box.gro -cs spc216.gro -p topol.top -o solvated.gro

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...
Generating solvent configuration
Will generate new solvent configuration of 4x4x6 boxes
Solvent box contains 51948 atoms in 17316 residues
Removed 4599 solvent atoms due to solvent-solvent overlap
Removed 4341 solvent atoms due to solute-solvent overlap
Sorting configuration
Found 1 molecule type:
    SOL (   3 atoms): 14336 residues
Generated solvent containing 43008 atoms in 14336 residues
Writing generated configuration to solvated.gro

Output configuration contains 47732 atoms in 14645 residues
Volume                 :     

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr -maxwarn 1

                      :-) GROMACS - gmx grompp, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr -maxwarn 1


NOTE 1 [file ions.mdp]:
  For a correct single-point energy evaluation with nsteps = 0, use
  continuation = yes to avoid constraining the input coordinates.

Setting the LD random seed to 1861081823

Generated 2926 of the 2926 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 2926 of the 2926 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

Excluding 3 bonded neighbours molecule type 'lig'

Excluding 2 bonded neighbours molecule type 'SOL'

NOTE 2 [file topol.top, line 44616]:
  System has non-zero total charge: -8.000000
  Total charge sh

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx genion -s ions.tpr -o ions.gro -p topol.top -pname NA -nname CL -neutral

                      :-) GROMACS - gmx genion, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx genion -s ions.tpr -o ions.gro -p topol.top -pname NA -nname CL -neutral

Reading file ions.tpr, VERSION 2025.3 (single precision)
Reading file ions.tpr, VERSION 2025.3 (single precision)
Will try to add 8 NA ions and 0 CL ions.
Select a continuous group of solvent molecules
Group     0 (         System) has 47732 elements
Group     1 (        Protein) has  4645 elements
Group     2 (      Protein-H) has  2296 elements
Group     3 (        C-alpha) has   308 elements
Group     4 (       Backbone) has   924 elements
Group     5 (      MainChain) has  1233 elements
Group     6 (   MainChain+Cb) has  1525 elements
Group     7 (    MainChain+H) has  1524 elements
Group     8 (      SideChain)

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx grompp -f min.mdp -c ions.gro -p topol.top -o em.tpr

                      :-) GROMACS - gmx grompp, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx grompp -f min.mdp -c ions.gro -p topol.top -o em.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file min.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Setting the LD random seed to -1153453383

Generated 2926 of the 2926 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 2926 of the 2926 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

Excluding 3 bonded neighbours molecule type 'lig'

Excluding 2 bonded neighbours molecule type 'SOL'

Excluding 1 bon

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -v -deffnm em

                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -v -deffnm em

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file em.tpr, VERSION 2025.3 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 12 OpenMP threads 


Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+03
   Number of steps    =        50000
Step=    0, Dmax= 1.0e-02 nm, Epot= -2.78190e+05 Fmax= 2.00080e+05, atom= 21519
Step=    1, Dmax= 1.0e-02 nm, Epot= -3.32139e+05 Fmax= 7.98628e+04, atom= 21519
Step=    2, Dmax= 1.2e-02 n

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx make_ndx -f em.gro -o index.ndx

                     :-) GROMACS - gmx make_ndx, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx make_ndx -f em.gro -o index.ndx


Reading structure file
Going to read 0 old index file(s)
Analysing residue names:
There are:   308    Protein residues
There are:     1      Other residues
There are: 14328      Water residues
There are:     8        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 47716 atoms
  1 Protein             :  4645 atoms
  2 Protein-H           :  2296 atoms
  3 C-alpha             :   308 atoms
  4 Backbone            :   924 atoms
  5 MainChain           :  1233 atoms
  6 MainChain+Cb        :  1525 atoms
  7 MainChain+H         :  1524 atoms
  8 SideChain     

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr -n index.ndx


                      :-) GROMACS - gmx grompp, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr -n index.ndx

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Setting the LD random seed to 1034284986

Generated 2926 of the 2926 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 2926 of the 2926 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'lig'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into 

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -v -deffnm nvt

                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -v -deffnm nvt

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file nvt.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.221 to 1.348

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 12 OpenMP threads 

starting mdrun 'Protein in wate

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -p topol.top -o npt.tpr -n index.ndx -maxwarn 1


                      :-) GROMACS - gmx grompp, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -p topol.top -o npt.tpr -n index.ndx -maxwarn 1

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

WARNING 1 [file npt.mdp]:
  The Berendsen barostat does not generate any strictly correct ensemble,
  and should not be used for new production simulations (in our opinion).
  We recommend using the C-rescale barostat instead.

Setting the LD random seed to -2038595585

Generated 2926 of the 2926 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 2926 of the 2926 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excl

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -v -deffnm npt

                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -v -deffnm npt

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file npt.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.221 to 1.348

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 12 OpenMP threads 

starting mdrun 'Protein in wate

In [ ]:
!ls

 ANTECHAMBER_AC.AC		  interaction_analysis.py   npt.xtc
 ANTECHAMBER_AC.AC0		  ions.gro		    nvt.cpt
 ANTECHAMBER_AM1BCC.AC		  ions.mdp		    nvt.edr
 ANTECHAMBER_AM1BCC_PRE.AC	  ions.tpr		    nvt.gro
 ANTECHAMBER_BOND_TYPE.AC	  lig.acpype		    nvt.log
 ANTECHAMBER_BOND_TYPE.AC0	  lig_fixed.mol2	    nvt.mdp
 ATOMTYPE.INF			  lig.frcmod		    nvt.tpr
 box.gro			  lig_GMX.gro		    nvt.xtc
 cgenff_charmm2gmx_py3_nx1.py	  lig_GMX.itp		    posre.itp
 complex_1h6u_hed_137633443.pdb   lig.mol2		    protein.gro
 complex.gro			  md.mdp		    protein.pdb
 em.edr				  mdout.mdp		    solvated.gro
 em.gro				  min.mdp		    sort_mol2_bonds.pl
 em.log				  npt.cpt		    sqm.in
 em.mdp				  npt.edr		    sqm.out
 em.tpr				  npt.gro		    sqm.pdb
 em.trr				  npt.log		    topol.top
'Gromacs Codes'			  npt.mdp		   '#topol.top.1#'
 index.ndx			  npt.tpr		   '#topol.top.2#'


1. Initial Production Setup ($0 \to 25 \text{ ns}$)

You need to use the final structure from your NPT run (npt.gro) and your final MD parameters (md.mdp) to prepare the first production run input file (md_0_25.tpr).

Files Needed: md.mdp, npt.gro, topol.top, index.ndx.

1.   Check md.mdp: Ensure your md.mdp file sets the first $25 \text{ ns}$ block.


*   integrator = md
*   dt = 0.002

*  nsteps = 12500000 ($25 \text{ ns}$)
*   continuation = no

*   nstxout-compressed = 5000 (If you want $10 \text{ ps}$ saving frequency, giving $2500$ frames per block).







In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx grompp -f md.mdp -c npt.gro -p topol.top -o md_0_25.tpr -n index.ndx -maxwarn 1

                      :-) GROMACS - gmx grompp, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx grompp -f md.mdp -c npt.gro -p topol.top -o md_0_25.tpr -n index.ndx -maxwarn 1

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Setting the LD random seed to 1608499117

Generated 2926 of the 2926 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 2926 of the 2926 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'lig'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds 

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -s md_0_25.tpr -deffnm md_0_25 -v -cpt 0.5

                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -s md_0_25.tpr -deffnm md_0_25 -v -cpt 0.5

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.221 to 1.348

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 12 OpenMP threads 




> This generates: md_0_25.xtc, md_0_25.edr, and md_0_25.cpt (for the next block).



2. Continuation Runs ($25 \text{ ns} \to 125 \text{ ns}$)
You will use the checkpoint file from the previous block to extend the run time for the next block.
Block 2: $25 \text{ ns} \to 50 \text{ ns}$


*  Generate $\text{TPR}$ for Block 2: Use the checkpoint from Block 1 (md_0_25.cpt) to continue.




In [ ]:
!date

Mon Oct 20 02:18:13 PM UTC 2025


In [ ]:
!cp md.mdp md2.mdp

In [ ]:
!ls

 ANTECHAMBER_AC.AC		  lig_fixed.mol2	  npt.edr
 ANTECHAMBER_AC.AC0		  lig.frcmod		  npt.gro
 ANTECHAMBER_AM1BCC.AC		  lig_GMX.gro		  npt.log
 ANTECHAMBER_AM1BCC_PRE.AC	  lig_GMX.itp		  npt.mdp
 ANTECHAMBER_BOND_TYPE.AC	  lig.mol2		  npt.tpr
 ANTECHAMBER_BOND_TYPE.AC0	  md_0_25.cpt		  npt.xtc
 ATOMTYPE.INF			  md_0_25.edr		  nvt.cpt
 box.gro			  md_0_25.gro		  nvt.edr
 cgenff_charmm2gmx_py3_nx1.py	  md_0_25.log		  nvt.gro
 complex_1h6u_hed_137633443.pdb   md_0_25_prev.cpt	  nvt.log
 complex.gro			  md_0_25.tpr		  nvt.mdp
 em.edr				  md_0_25.xtc		  nvt.tpr
 em.gro				  md_25_50.part0002.edr   nvt.xtc
 em.log				  md_25_50.part0002.gro   posre.itp
 em.mdp				  md_25_50.part0002.log   protein.gro
 em.tpr				  md_25_50.part0002.xtc   protein.pdb
 em.trr				  md_25_50.tpr		  solvated.gro
'Gromacs Codes'			 '#md_25_50.tpr.1#'	  sort_mol2_bonds.pl
 index.ndx			 '#md_25_50.tpr.2#'	  sqm.in
 interaction_analysis.py	  md2.mdp		  sqm.out
 ions.gro			  md.mdp		  sqm.pdb
 ions.mdp			  mdout.mdp		  

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx convert-tpr -s md_0_25.tpr -o md_25_50.tpr -extend 25000

                   :-) GROMACS - gmx convert-tpr, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx convert-tpr -s md_0_25.tpr -o md_25_50.tpr -extend 25000

Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Input file:
  Run start step                                     0     
  Run start time                                     0 ps  
  Step to be made during run                  12500000     
  Runtime for the run                            25000 ps  
  Run end step                                12500000     
  Run end time                                   25000 ps  

Extending remaining runtime by 25000 ps

Output file:
  Run start step                                     0     
  Run start time                                     0 ps  
  Step to be made during run  

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -s md_25_50.tpr -deffnm md_25_50 -v -cpt 0.5 -cpi md_0_25.cpt -noappend

                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -s md_25_50.tpr -deffnm md_25_50 -v -cpt 0.5 -cpi md_0_25.cpt -noappend


Back Off! I just backed up md_25_50.part0002.log to ./#md_25_50.part0002.log.1#
Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file md_25_50.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.221 to 1.348

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constra

To resume successfully, you must tell GROMACS to use the exact output file names recorded in the checkpoint. Since your run was labeled part0002, you need to set the output prefix to include that part number.

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -s md_25_50.tpr -deffnm md_25_50.part0002 -v -cpt 0.5 -cpi md_25_50.cpt

                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -s md_25_50.tpr -deffnm md_25_50.part0002 -v -cpt 0.5 -cpi md_25_50.cpt

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file md_25_50.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.221 to 1.348

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI th

This tool checks the integrity of the compressed trajectory file (.xtc) and reports the total frames and the time interval between them.
For the Block 2 run (which completed the $25 \text{ ns} \to 50 \text{ ns}$ segment), you would run:

In [ ]:
!ls

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx trjcat \
-f md_25_50.part0002.xtc md_25_50.part0003.xtc \
-o md_25_50.xtc

                      :-) GROMACS - gmx trjcat, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx trjcat -f md_25_50.part0002.xtc md_25_50.part0003.xtc -o md_25_50.xtc

Reading frame       1 time 42290.000   

Summary of files and start times used:

          File                Start time       Time step
---------------------------------------------------------
    md_25_50.part0002.xtc    25000.000 ps       10.000 ps
    md_25_50.part0003.xtc    42280.000 ps       10.000 ps


Back Off! I just backed up md_25_50.xtc to ./#md_25_50.xtc.1#
Reading frame       0 time 25000.000   Note that major changes are planned in future for trjcat, to improve usability and utility.
lasttime 0

Continue writing frames from md_25_50.part0002.xtc t=25000 ps, frame=0      
Reading frame    1700 time 420

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx eneconv \
-f md_25_50.part0002.edr md_25_50.part0003.edr \
-o md_25_50.edr

                     :-) GROMACS - gmx eneconv, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx eneconv -f md_25_50.part0002.edr md_25_50.part0003.edr -o md_25_50.edr

Opened md_25_50.part0002.edr as single precision energy file
Reading energy frame      1 time 25010.000         
Opened md_25_50.part0003.edr as single precision energy file
Reading energy frame      0 time 42280.000         

Summary of files and start times used:

          File                Start time
-----------------------------------------
    md_25_50.part0002.edr    25000.000
    md_25_50.part0003.edr    42280.000

Opened md_25_50.part0002.edr as single precision energy file
Reading energy frame      0 time 25000.000         
Continue writing frames from t=25000, step=0
Last energy frame read 1728 time 42271

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx check -f md_25_50.xtc

                      :-) GROMACS - gmx check, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx check -f md_25_50.xtc

Checking file md_25_50.xtc
Reading frame       0 time 25000.000   
# Atoms  47716
Precision 0.001 (nm)
Reading frame    2000 time 45000.000   


Item        #frames Timestep (ps)
Step          2501    10
Time          2501    10
Lambda           0
Coords        2501    10
Velocities       0
Forces           0
Box           2501    10

GROMACS reminds you: "A good sign these days when you're listening to someone talk about this epidemic is the number of times they say 'We don't know yet'. The more of those, the better." (Derek Lowe)



Expected Output Check


*   Total Frames: Should be $2,500$ frames (based on a $10 \text{ ps}$ saving interval for $25,000 \text{ ps}$).
*   Time: Should range from $25,000.0 \text{ ps}$ to $50,000.0 \text{ ps}$.

*   Timestep: Should be $10 \text{ ps}$.





Block 3: $50 \text{ ns} \to 75 \text{ ns}$


*   Generate $\text{TPR}$ for Block 3: Use the checkpoint from Block 2 (md_25_50.cpt).



In [ ]:
!date

Mon Oct 20 04:46:50 PM UTC 2025


In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx trjcat \
-f md_25_50.part0002.xtc md_25_50.part0003.xtc \
-o md_25_50_temp.xtc

                      :-) GROMACS - gmx trjcat, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx trjcat -f md_25_50.part0002.xtc md_25_50.part0003.xtc -o md_25_50_temp.xtc

Reading frame       1 time 42290.000   

Summary of files and start times used:

          File                Start time       Time step
---------------------------------------------------------
    md_25_50.part0002.xtc    25000.000 ps       10.000 ps
    md_25_50.part0003.xtc    42280.000 ps       10.000 ps

Reading frame       0 time 25000.000   Note that major changes are planned in future for trjcat, to improve usability and utility.
lasttime 0

Continue writing frames from md_25_50.part0002.xtc t=25000 ps, frame=0      
Reading frame    1700 time 42000.000    ->  frame   1700 time 42000.000 ps     
Reading 

In [ ]:
!mv md_25_50_temp.xtc md_25_50.xtc

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx convert-tpr -s md_0_25.tpr -o md_50_75.tpr -extend 50000

                   :-) GROMACS - gmx convert-tpr, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx convert-tpr -s md_0_25.tpr -o md_50_75.tpr -extend 50000

Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Input file:
  Run start step                                     0     
  Run start time                                     0 ps  
  Step to be made during run                  12500000     
  Runtime for the run                            25000 ps  
  Run end step                                12500000     
  Run end time                                   25000 ps  

Extending remaining runtime by 50000 ps

Output file:
  Run start step                                     0     
  Run start time                                     0 ps  
  Step to be made during run  

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -s md_50_75.tpr -deffnm md_50_75.part0003 -v -cpt 0.5 -cpi md_50_75.cpt

                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -s md_50_75.tpr -deffnm md_50_75.part0003 -v -cpt 0.5 -cpi md_50_75.cpt

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file md_50_75.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.221 to 1.348

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI th

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx mdrun -s md_50_75.tpr -cpi md_50_75.part0003.cpt -deffnm md_50_75.part0004 -v -cpt 0.5

                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -s md_50_75.tpr -cpi md_50_75.part0003.cpt -deffnm md_50_75.part0004 -v -cpt 0.5

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file md_50_75.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.22 to 1.347

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx trjcat -f md_50_75.part0003.xtc md_50_75.part0004.xtc -o md_50_75_combined.xtc -settime

                      :-) GROMACS - gmx trjcat, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx trjcat -f md_50_75.part0003.xtc md_50_75.part0004.xtc -o md_50_75_combined.xtc -settime

Reading frame       0 time 42280.000   ^C


Block 4: $75 \text{ ns} \to 100 \text{ ns}$

*   Generate $\text{TPR}$ for Block 4: Use the checkpoint from Block 3 (md_50_75.cpt).



In [ ]:
!ls


 ANTECHAMBER_AC.AC		  md_50_75.part0003.log
 ANTECHAMBER_AC.AC0		 '#md_50_75.part0003.log.1#'
 ANTECHAMBER_AM1BCC.AC		 '#md_50_75.part0003.log.2#'
 ANTECHAMBER_AM1BCC_PRE.AC	  md_50_75.part0003_prev.cpt
 ANTECHAMBER_BOND_TYPE.AC	  md_50_75.part0003.xtc
 ANTECHAMBER_BOND_TYPE.AC0	 '#md_50_75.part0003.xtc.1#'
 ATOMTYPE.INF			  md_50_75.part0004.edr
 box.gro			  md_50_75.part0004.gro
 cgenff_charmm2gmx_py3_nx1.py	  md_50_75.part0004.log
 complex_1h6u_hed_137633443.pdb   md_50_75.part0004.xtc
 complex.gro			  md_50_75.part0005.edr
 em.edr				  md_50_75.part0005.gro
 em.gro				  md_50_75.part0005.log
 em.log				  md_50_75.part0005.xtc
 em.mdp				  md_50_75_prev.cpt
 em.tpr				  md_50_75.tpr
 em.trr				 '#md_50_75.tpr.1#'
 energy.xvg			 '#md_50_75.tpr.2#'
'Gromacs Codes'			  md_50_75.xtc
 index.ndx			  md_75_100.cpt
 interaction_analysis.py	  md_75_100.part0001.cpt
 ions.gro			  md_75_100.part0001.edr
 ions.mdp			  md_75_100.part0001.log
 ions.tpr			  md_75_100.part0001_prev.cpt
 lig.acpype	

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx trjcat \
-f md_50_75.part0003.xtc md_50_75.part0004.xtc md_50_75.part0005.xtc \
-o md_50_75.xtc

                      :-) GROMACS - gmx trjcat, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx trjcat -f md_50_75.part0003.xtc md_50_75.part0004.xtc md_50_75.part0005.xtc -o md_50_75.xtc

Last frame          0 time 75000.000   


Summary of files and start times used:

          File                Start time       Time step
---------------------------------------------------------
    md_50_75.part0003.xtc    42280.000 ps       10.000 ps
    md_50_75.part0004.xtc    42620.000 ps       10.000 ps
    md_50_75.part0005.xtc    75000.000 ps        0.000 ps

Reading frame       0 time 42280.000   Note that major changes are planned in future for trjcat, to improve usability and utility.
lasttime 0

Continue writing frames from md_50_75.part0003.xtc t=42280 ps, frame=0      
Reading fram

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx eneconv \
-f md_50_75.part0003.edr md_50_75.part0004.edr md_50_75.part0005.edr \
-o md_50_75.edr

                     :-) GROMACS - gmx eneconv, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx eneconv -f md_50_75.part0003.edr md_50_75.part0004.edr md_50_75.part0005.edr -o md_50_75.edr

Opened md_50_75.part0003.edr as single precision energy file
Reading energy frame      1 time 42290.000         
Opened md_50_75.part0004.edr as single precision energy file
Reading energy frame      0 time 42620.000         
Opened md_50_75.part0005.edr as single precision energy file
Reading energy frame      0 time 75000.000         

Summary of files and start times used:

          File                Start time
-----------------------------------------
    md_50_75.part0003.edr    42280.000
    md_50_75.part0004.edr    42620.000
    md_50_75.part0005.edr    75000.000

Opened md_50_75.part00

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx trjcat \
-f md_0_25.xtc md_25_50.xtc md_50_75.xtc \
-o md_0_75.xtc

                      :-) GROMACS - gmx trjcat, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx trjcat -f md_0_25.xtc md_25_50.xtc md_50_75.xtc -o md_0_75.xtc

Reading frame       1 time 42290.000   

Summary of files and start times used:

          File                Start time       Time step
---------------------------------------------------------
              md_0_25.xtc        0.000 ps       10.000 ps
             md_25_50.xtc    25000.000 ps       10.000 ps
             md_50_75.xtc    42280.000 ps       10.000 ps

Reading frame       0 time    0.000   Note that major changes are planned in future for trjcat, to improve usability and utility.
lasttime 0

Continue writing frames from md_0_25.xtc t=0 ps, frame=0      

Reading frame       0 time 25000.000   
lasttime 24990



In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx eneconv \
-f md_0_25.edr md_25_50.edr md_50_75.edr \
-o md_0_75.edr

                     :-) GROMACS - gmx eneconv, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx eneconv -f md_0_25.edr md_25_50.edr md_50_75.edr -o md_0_75.edr

Opened md_0_25.edr as single precision energy file
Reading energy frame      1 time   10.000         
Opened md_25_50.edr as single precision energy file
Reading energy frame      0 time 25000.000         
Opened md_50_75.edr as single precision energy file
Reading energy frame      0 time 42280.000         

Summary of files and start times used:

          File                Start time
-----------------------------------------
              md_0_25.edr        0.000
             md_25_50.edr    25000.000
             md_50_75.edr    42280.000

Opened md_0_25.edr as single precision energy file
Reading energy frame      0 t

In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx check -f md_0_75.xtc

                      :-) GROMACS - gmx check, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx check -f md_0_75.xtc

Checking file md_0_75.xtc
Reading frame       0 time    0.000   
# Atoms  47716
Precision 0.001 (nm)
Reading frame    7000 time 70000.000   


Item        #frames Timestep (ps)
Step          7501    10
Time          7501    10
Lambda           0
Coords        7501    10
Velocities       0
Forces           0
Box           7501    10

GROMACS reminds you: "You should call it 'entropy'. No one knows what entropy really is, so in a debate you will always have the advantage." (John von Neumann to Claude Shannon, on why he should borrow the term for information theory)



In [ ]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx energy -f md_0_75.edr

                      :-) GROMACS - gmx energy, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx energy -f md_0_75.edr

Opened md_0_75.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Coulomb-(SR)  
  9  Coul.-recip.    10  Potential       11  Kinetic-En.     12  Total-Energy  
 13  Conserved-En.   14  Temperature     15  Pressure        16  Constr.-rmsd  
 17  Box-X           18  Box-Y           19  Box-Z           20 

In [ ]:
import os

print("--- 1. Cleaning up corrupted files ---")

# List of bad files to delete
bad_files = [
    # Corrupted 25-50 ns run (parts and combined)
    "md_25_50.part0002.cpt", "md_25_50.part0002.edr", "md_25_50.part0002.log", "md_25_50.part0002.xtc",
    "md_25_50.part0003.cpt", "md_25_50.part0003.edr", "md_25_50.part0003.log", "md_25_50.part0003.xtc",
    "md_25_50.cpt", "md_25_50.edr", "md_25_50.xtc",

    # Corrupted 50-75 ns run (parts and combined)
    "md_50_75.part0003.cpt", "md_50_75.part0003.edr", "md_50_75.part0003.log", "md_50_75.part0003.xtc",
    "md_50_75.part0004.cpt", "md_50_75.part0004.edr", "md_50_75.part0004.log", "md_50_75.part0004.xtc",
    "md_50_75.part0005.cpt", "md_50_75.part0005.edr", "md_50_75.part0005.log", "md_50_75.part0005.xtc",
    "md_50_75.cpt", "md_50_75.edr", "md_50_75.xtc",

    # Corrupted 75-100 ns run (parts)
    "md_75_100.part0001.cpt", "md_75_100.part0001.edr", "md_75_100.part0001.log", "md_75_100.part0001.xtc",
    "md_75_100.part0002.cpt", "md_75_100.part0002.edr", "md_75_100.part0002.log", "md_75_100.part0002.xtc",
    "md_75_100.part0004.edr", "md_75_100.part0004.log", "md_75_100.part0004.xtc",

    # Corrupted combined 0-75 ns files
    "md_0_75.edr", "md_0_75.xtc"
]

for f in bad_files:
    if os.path.exists(f):
        os.remove(f)
        print(f"Removed: {f}")
    else:
        print(f"Not found (already deleted): {f}")

print("\n--- ✅ Cleanup Finished ---")

--- 1. Cleaning up corrupted files ---
Removed: md_25_50.part0002.cpt
Removed: md_25_50.part0002.edr
Removed: md_25_50.part0002.log
Removed: md_25_50.part0002.xtc
Not found (already deleted): md_25_50.part0003.cpt
Removed: md_25_50.part0003.edr
Removed: md_25_50.part0003.log
Removed: md_25_50.part0003.xtc
Removed: md_25_50.cpt
Removed: md_25_50.edr
Removed: md_25_50.xtc
Removed: md_50_75.part0003.cpt
Removed: md_50_75.part0003.edr
Removed: md_50_75.part0003.log
Removed: md_50_75.part0003.xtc
Not found (already deleted): md_50_75.part0004.cpt
Removed: md_50_75.part0004.edr
Removed: md_50_75.part0004.log
Removed: md_50_75.part0004.xtc
Not found (already deleted): md_50_75.part0005.cpt
Removed: md_50_75.part0005.edr
Removed: md_50_75.part0005.log
Removed: md_50_75.part0005.xtc
Removed: md_50_75.cpt
Removed: md_50_75.edr
Removed: md_50_75.xtc
Removed: md_75_100.part0001.cpt
Removed: md_75_100.part0001.edr
Removed: md_75_100.part0001.log
Removed: md_75_100.part0001.xtc
Not found (already de

In [ ]:
print("--- 2. Re-running Block 2 (25-50 ns) - Correctly ---")
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && \
gmx mdrun -s md_25_50.tpr -deffnm md_25_50 -v -cpt 0.5 -cpi md_0_25.cpt -noappend

--- 2. Re-running Block 2 (25-50 ns) - Correctly ---
                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -s md_25_50.tpr -deffnm md_25_50 -v -cpt 0.5 -cpi md_0_25.cpt -noappend

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file md_25_50.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.221 to 1.348

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PM

In [ ]:
!!ls -lt *.cpt

['-rw------- 1 root root 1147492 Oct 20 22:40 md_25_50.cpt',
 '-rw------- 1 root root 1147492 Oct 20 22:40 md_25_50_prev.cpt',
 '-rw------- 1 root root 1147492 Oct 20 20:11 md_75_100.part0001_prev.cpt',
 '-rw------- 1 root root 1146684 Oct 20 20:04 md_75_100.cpt',
 '-rw------- 1 root root 1147492 Oct 20 20:04 md_75_100_prev.cpt',
 '-rw------- 1 root root 1147492 Oct 20 19:37 md_50_75.part0003_prev.cpt',
 '-rw------- 1 root root 1147492 Oct 20 16:54 md_50_75_prev.cpt',
 '-rw------- 1 root root 1147492 Oct 20 16:43 md_25_50.part0002_prev.cpt',
 '-rw------- 1 root root 1147456 Oct 20 14:02 md_0_25.cpt',
 '-rw------- 1 root root 1147456 Oct 20 14:02 md_0_25_prev.cpt',
 '-rw------- 1 root root 1147324 Oct 20 11:36 npt.cpt',
 '-rw------- 1 root root 1147096 Oct 20 11:31 nvt.cpt']

In [ ]:
# --- 3b. Run Block 3 (50-75 ns) ---
print("--- 3. Running Block 3 (50-75 ns) ---")

# Using the correct checkpoint file "md_25_50.cpt"
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && \
gmx mdrun -s md_50_75.tpr -deffnm md_50_75 -v -cpt 0.5 -cpi md_25_50.cpt -noappend

--- 3. Running Block 3 (50-75 ns) ---
                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -s md_50_75.tpr -deffnm md_50_75 -v -cpt 0.5 -cpi md_25_50.cpt -noappend

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file md_50_75.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.22 to 1.347

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do

Step 4: Run Block 4 (75 ns $\to$ 100 ns)

In [ ]:
!ls -lt *.cpt


-rw------- 1 root root 1147492 Oct 21 00:55 md_50_75.cpt
-rw------- 1 root root 1147492 Oct 21 00:55 md_50_75_prev.cpt
-rw------- 1 root root 1147492 Oct 20 22:40 md_25_50.cpt
-rw------- 1 root root 1147492 Oct 20 22:40 md_25_50_prev.cpt
-rw------- 1 root root 1147492 Oct 20 20:11 md_75_100.part0001_prev.cpt
-rw------- 1 root root 1146684 Oct 20 20:04 md_75_100.cpt
-rw------- 1 root root 1147492 Oct 20 20:04 md_75_100_prev.cpt
-rw------- 1 root root 1147492 Oct 20 19:37 md_50_75.part0003_prev.cpt
-rw------- 1 root root 1147492 Oct 20 16:43 md_25_50.part0002_prev.cpt
-rw------- 1 root root 1147456 Oct 20 14:02 md_0_25.cpt
-rw------- 1 root root 1147456 Oct 20 14:02 md_0_25_prev.cpt
-rw------- 1 root root 1147324 Oct 20 11:36 npt.cpt
-rw------- 1 root root 1147096 Oct 20 11:31 nvt.cpt


In [ ]:
# --- 4b. Run Block 4 (75-100 ns) ---
print("--- 4. Running Block 4 (75-100 ns) ---")

# Using the correct checkpoint file "md_50_75.cpt"
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && \
gmx mdrun -s md_75_100.tpr -deffnm md_75_100 -v -cpt 0.5 -cpi md_50_75.cpt -noappend

--- 4. Running Block 4 (75-100 ns) ---
                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -s md_75_100.tpr -deffnm md_75_100 -v -cpt 0.5 -cpi md_50_75.cpt -noappend

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file md_75_100.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.221 to 1.348

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks wi

In [ ]:
!date
!ls -lt *.cpt

Tue Oct 21 07:43:27 AM UTC 2025
-rw------- 1 root root 1147492 Oct 21 07:37 md_75_100.cpt
-rw------- 1 root root 1147492 Oct 21 07:37 md_75_100_prev.cpt
-rw------- 1 root root 1147492 Oct 21 00:55 md_50_75.cpt
-rw------- 1 root root 1147492 Oct 21 00:55 md_50_75_prev.cpt
-rw------- 1 root root 1147492 Oct 20 22:40 md_25_50.cpt
-rw------- 1 root root 1147492 Oct 20 22:40 md_25_50_prev.cpt
-rw------- 1 root root 1147492 Oct 20 20:11 md_75_100.part0001_prev.cpt
-rw------- 1 root root 1147492 Oct 20 19:37 md_50_75.part0003_prev.cpt
-rw------- 1 root root 1147492 Oct 20 16:43 md_25_50.part0002_prev.cpt
-rw------- 1 root root 1147456 Oct 20 14:02 md_0_25.cpt
-rw------- 1 root root 1147456 Oct 20 14:02 md_0_25_prev.cpt
-rw------- 1 root root 1147324 Oct 20 11:36 npt.cpt
-rw------- 1 root root 1147096 Oct 20 11:31 nvt.cpt


In [ ]:
# --- 5a. Create TPR for Block 5 ---
print("--- 5a. Creating TPR file for 100-125 ns block ---")

!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && \
gmx convert-tpr -s md_0_25.tpr -o md_100_125.tpr -extend 100000.0

--- 5a. Creating TPR file for 100-125 ns block ---
                   :-) GROMACS - gmx convert-tpr, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx convert-tpr -s md_0_25.tpr -o md_100_125.tpr -extend 100000.0

Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Input file:
  Run start step                                     0     
  Run start time                                     0 ps  
  Step to be made during run                  12500000     
  Runtime for the run                            25000 ps  
  Run end step                                12500000     
  Run end time                                   25000 ps  

Extending remaining runtime by 100000 ps

Output file:
  Run start step                                     0     
  Run start time                 

In [ ]:
# --- 5b. Run Block 5 (100-125 ns) ---
print("--- 5b. Running FINAL Block 5 (100-125 ns) ---")

# Using the correct checkpoint file "md_75_100.cpt"
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && \
gmx mdrun -s md_100_125.tpr -deffnm md_100_125 -v -cpt 0.5 -cpi md_75_100.cpt -noappend

--- 5b. Running FINAL Block 5 (100-125 ns) ---
                      :-) GROMACS - gmx mdrun, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx mdrun -s md_100_125.tpr -deffnm md_100_125 -v -cpt 0.5 -cpi md_75_100.cpt -noappend

Compiled SIMD is AVX2_256, but CPU also supports AVX_512 (see log).
Reading file md_100_125.tpr, VERSION 2025.3 (single precision)
Changing nstlist from 20 to 100, rlist from 1.221 to 1.348

Update groups can not be used for this system because atoms that are (in)directly constrained together are interdispersed with other atoms

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU


In [11]:
!ls

 ANTECHAMBER_AC.AC		 '#md_25_50.xtc.1#'
 ANTECHAMBER_AC.AC0		  md2.mdp
 ANTECHAMBER_AM1BCC.AC		  md_50_75.cpt
 ANTECHAMBER_AM1BCC_PRE.AC	  md_50_75.part0003.edr
 ANTECHAMBER_BOND_TYPE.AC	 '#md_50_75.part0003.edr.1#'
 ANTECHAMBER_BOND_TYPE.AC0	  md_50_75.part0003.gro
 ATOMTYPE.INF			  md_50_75.part0003.log
 box.gro			 '#md_50_75.part0003.log.1#'
 cgenff_charmm2gmx_py3_nx1.py	 '#md_50_75.part0003.log.2#'
 complex_1h6u_hed_137633443.pdb   md_50_75.part0003_prev.cpt
 complex.gro			  md_50_75.part0003.xtc
 em.edr				 '#md_50_75.part0003.xtc.1#'
 em.gro				  md_50_75.part0004.gro
 em.log				  md_50_75.part0005.gro
 em.mdp				  md_50_75_prev.cpt
 em.tpr				  md_50_75.tpr
 em.trr				 '#md_50_75.tpr.1#'
 energy.xvg			 '#md_50_75.tpr.2#'
'Gromacs Codes'			  md_75_100.cpt
 index.ndx			  md_75_100.part0001_prev.cpt
 interaction_analysis.py	 '#md_75_100.part0002.edr.1#'
 ions.gro			 '#md_75_100.part0002.log.1#'
 ions.mdp			 '#md_75_100.part0002.xtc.1#'
 ions.tpr			  md_75_100.part0004.edr
 lig.acpy

In [12]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx trjcat -f md_0_25.xtc md_25_50.part0002.xtc md_50_75.part0003.xtc md_75_100.part0004.xtc md_100_125.part0005.xtc -o md_0_125_combined.xtc

                      :-) GROMACS - gmx trjcat, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx trjcat -f md_0_25.xtc md_25_50.part0002.xtc md_50_75.part0003.xtc md_75_100.part0004.xtc md_100_125.part0005.xtc -o md_0_125_combined.xtc

Reading frame       1 time 100010.000   

Summary of files and start times used:

          File                Start time       Time step
---------------------------------------------------------
              md_0_25.xtc        0.000 ps       10.000 ps
    md_25_50.part0002.xtc    25000.000 ps       10.000 ps
    md_50_75.part0003.xtc    50000.000 ps       10.000 ps
   md_75_100.part0004.xtc    75000.000 ps       10.000 ps
  md_100_125.part0005.xtc   100000.000 ps       10.000 ps

Reading frame       0 time    0.000   Note that major changes are plan

In [13]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && echo "Protein_LIG System" | gmx trjconv -s md_0_25.tpr -f md_0_125_combined.xtc -n index.ndx -o md_0_125_center.xtc -center -pbc whole

                     :-) GROMACS - gmx trjconv, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx trjconv -s md_0_25.tpr -f md_0_125_combined.xtc -n index.ndx -o md_0_125_center.xtc -center -pbc whole

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Select group for centering
Group     0 (         System) has 47716 elements
Group     1 (        Protein) has  4645 elements
Group     2 (      Protein-H) has  2296 elements
Group     3 (        C-alpha) has   308 elements
Group     4 (       Backbone) has   924 elements
Group     5 (      MainChain) has  1233 ele

In [14]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && echo "Protein_LIG Protein_LIG" | gmx trjconv -s md_0_25.tpr -f md_0_125_center.xtc -n index.ndx -o md_0_125_final.xtc -fit rot+trans

                     :-) GROMACS - gmx trjconv, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx trjconv -s md_0_25.tpr -f md_0_125_center.xtc -n index.ndx -o md_0_125_final.xtc -fit rot+trans

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Reading file md_0_25.tpr, VERSION 2025.3 (single precision)
Select group for least squares fit
Group     0 (         System) has 47716 elements
Group     1 (        Protein) has  4645 elements
Group     2 (      Protein-H) has  2296 elements
Group     3 (        C-alpha) has   308 elements
Group     4 (       Backbone) has   924 elements
Group     5 (      MainChain) has  1233 el

In [15]:
!source /content/drive/MyDrive/Colab_Software/GROMACS/gromacs_env.sh && gmx eneconv -f md_0_25.edr md_25_50.part0002.edr md_50_75.part0003.edr md_75_100.part0004.edr md_100_125.part0005.edr -o md.edr

                     :-) GROMACS - gmx eneconv, 2025.3 (-:

Executable:   /content/drive/MyDrive/Colab_Software/GROMACS/bin/gmx
Data prefix:  /content/drive/MyDrive/Colab_Software/GROMACS
Working dir:  /content/drive/MyDrive/MANIT Internship Project/MD_Simulation_AMBERFF/hedragenin
Command line:
  gmx eneconv -f md_0_25.edr md_25_50.part0002.edr md_50_75.part0003.edr md_75_100.part0004.edr md_100_125.part0005.edr -o md.edr

Opened md_0_25.edr as single precision energy file
Reading energy frame      1 time   10.000         
Opened md_25_50.part0002.edr as single precision energy file
Reading energy frame      0 time 25000.000         
Opened md_50_75.part0003.edr as single precision energy file
Reading energy frame      0 time 50000.000         
Opened md_75_100.part0004.edr as single precision energy file
Reading energy frame      0 time 75000.000         
Opened md_100_125.part0005.edr as single precision energy file
Reading energy frame      0 time 100000.000         

Summary of fi

In [16]:
# Rename your final, clean trajectory
!mv md_0_125_final.xtc md.xtc

# Rename your final structure (from the 125 ns run)
!mv md_100_125.part0005.gro md.gro

In [17]:
!ls


 ANTECHAMBER_AC.AC		  md2.mdp
 ANTECHAMBER_AC.AC0		  md_50_75.cpt
 ANTECHAMBER_AM1BCC.AC		  md_50_75.part0003.edr
 ANTECHAMBER_AM1BCC_PRE.AC	 '#md_50_75.part0003.edr.1#'
 ANTECHAMBER_BOND_TYPE.AC	  md_50_75.part0003.gro
 ANTECHAMBER_BOND_TYPE.AC0	  md_50_75.part0003.log
 ATOMTYPE.INF			 '#md_50_75.part0003.log.1#'
 box.gro			 '#md_50_75.part0003.log.2#'
 cgenff_charmm2gmx_py3_nx1.py	  md_50_75.part0003_prev.cpt
 complex_1h6u_hed_137633443.pdb   md_50_75.part0003.xtc
 complex.gro			 '#md_50_75.part0003.xtc.1#'
 em.edr				  md_50_75.part0004.gro
 em.gro				  md_50_75.part0005.gro
 em.log				  md_50_75_prev.cpt
 em.mdp				  md_50_75.tpr
 em.tpr				 '#md_50_75.tpr.1#'
 em.trr				 '#md_50_75.tpr.2#'
 energy.xvg			  md_75_100.cpt
'Gromacs Codes'			  md_75_100.part0001_prev.cpt
 index.ndx			 '#md_75_100.part0002.edr.1#'
 interaction_analysis.py	 '#md_75_100.part0002.log.1#'
 ions.gro			 '#md_75_100.part0002.xtc.1#'
 ions.mdp			  md_75_100.part0004.edr
 ions.tpr			  md_75_100.part0004.gro
 lig

In [ ]:
print("MD run has finished.")
print("Shutting down Colab runtime to save points...")

from google.colab import runtime
runtime.unassign()

MD run has finished.
Shutting down Colab runtime to save points...
